In [ ]:
%matplotlib inline

# import modules
import sys
import os
# import xarray as xr
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
# import netCDF4 as nc
# import time
# import mpl_toolkits.basemap
# import cartopy.crs as ccrs
# import cartopy.feature as cfeature
from mpl_toolkits.basemap import Basemap
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt
# import matplotlib.colors as colors

# from matplotlib import path
# from cmocean import cm

modpath = os.path.abspath(os.path.join('/home/chrenkl/Projects/nemo_tools/src'))
if modpath not in sys.path:
    sys.path.append(modpath)
import nemo_tools as nt

modpath = os.path.abspath(os.path.join('/home/chrenkl/Projects/nemo_bathymetry/src'))
if modpath not in sys.path:
    sys.path.append(modpath)

from create_bathymetry import read_insitu, \
                              read_webtide                            

In [ ]:
# project directory
projdir = '/home/chrenkl/Projects/nemo_bathymetry'

# NEMO configuration
conf = 'MBnest'
    
# WebTide data set
wtset = 'nwatl'

# tidal constituents for datum correction and minimum depth
constituents = ['M2', 'S2', 'N2', 'K1', 'O1']

# create directory for figures
figdir = os.path.join(projdir, 'reports/figures', conf)

if not os.path.exists(figdir):
        os.makedirs(figdir)

## Data Preparation

For the gridding of the bathymetry observations we need the NEMO grid coordinates and the observations itself.

In [ ]:
# NEMO grid ---------------------------------------------------
NEMO = nt.nemo_grid(os.path.join(projdir, 'data/raw', conf,
                                 'Configuration',
                                 'coordinates.nc'))

# create map projection for interpolations
proj = Basemap(projection='merc',
               llcrnrlat=NEMO['latmin'], urcrnrlat=NEMO['latmax'],
               llcrnrlon=NEMO['lonmin'], urcrnrlon=NEMO['lonmax'],
               resolution='h')
    
# bathymetry observations -------------------------------------

### >>> for now just read in the cleaned up data <<<
LLWLT = pd.read_csv(os.path.join(projdir, 'data/interim', 'LLWLTData.lld'),
                                 header=None,
                                 delimiter='\s+',
                                 names=['lon', 'lat', 'depth'],
                                 engine='python')

MSL = pd.read_csv(os.path.join(projdir, 'data/interim', 'MSLData.lld'),
                               header=None,
                               delimiter='\s+',
                               names=['lon', 'lat', 'depth'],
                               engine='python')

nllwt = len(LLWLT.index)

### Correct Vertical Datum

Some of the observations are referenced to Mean Sea Level (MSL) while others are with respect to Lower Low Water Large Tides (LLWLT). The goal is to unify the vertical datum of the observations. Since gridded bathymetry products like GEBCO or ETOPO1 are typically referenced to MSL, it makes sense to use it as the common vertical datum for all data sets.

We correct all observations referenced to LLWLT to MSL using WebTide.

In [ ]:
# WebTide data
WebTide = read_webtide(os.path.join(projdir, 'data/external/WebTide'),
                       wtset,
                       constituents=constituents,
                       ampmax=True)

In [ ]:
# file name of cleaned and corrected insitu data
insitufile = os.path.join(projdir,
                          'data/interim',
                          '%s_in-situ_corrected.h5' % conf)

try:

    InSitu = pd.read_hdf(insitufile, 'InSitu')
    print('Found in-situ_corrected.h5. Skip vertical datum correction.')

except:

    print('Correct vertical datum of in-situ data using WebTide.')

    # source coordinates: WebTide
    xs, ys = proj(WebTide.lon.values, WebTide.lat.values)

    # target coordinates: in-situ observations
    xt, yt = proj(LLWLT.lon.values, LLWLT.lat.values)

    # create cKDTree object to represent source grid
    kdt = cKDTree(list(zip(xs, ys)))

    # find indices of the nearest grid point in the flattened array
    dist, inds = kdt.query(list(zip(xt, yt)), k = 1)   

    # initialize
    LLWLTMSL = LLWLT.drop('depth', axis=1)
    LLWLTMSL['depth'] = np.zeros(nllwt)

    # from Fatemeh's code
    ffact = .85 + .15 * (WebTide.ampmax[inds].values - 2.) / 3.
    ffact[WebTide.ampmax[inds].values <= 2.] = .85
    ffact[WebTide.ampmax[inds].values >= 5.] = 1.

    # correction (negative down)
    LLWLTMSL.depth = LLWLT.depth - ffact * WebTide.ampmax[inds].values

    # concatenate data sets
    InSitu = pd.concat([MSL, LLWLTMSL], ignore_index=True)

    # release memory
    # del LLWLT, LLWLTMSL, MSL

    # save cleaned and corrected data set
    store = pd.HDFStore(insitufile)   
    store['InSitu'] = InSitu    
    store.close()

## Gridding - Barnes Algorithm

In [ ]:
def update_point(xp, yp, valp, xobs, yobs, values, gamma=1., kappa=None):
    
    # compute squared distance
    dist2 = (xp - xobs)**2 + (yp - yobs)**2

    # Barnes weights
    wm = np.exp(-1. * dist2 / (gamma * kappa))

    # compute update
    update = np.sum(wm * np.subtract(values, valp) / wm.sum())

    # compute gridded variable
    return valp + update

def calc_kappa(delta, kappa_star=5.052):

    return kappa_star * (2. * delta / np.pi)**2

def barnes_gridding(NEMO, obs, proj, sradius, gamma):
    
    # convert lat/lon of NEMO grid and observations to Cartesian coordinates
    xm, ym = proj(NEMO.glamt.values.ravel(), NEMO.gphit.values.ravel())
    xo, yo = proj(InSitu.lon.values, InSitu.lat.values)

    # create cKDTree of NEMO grid and observations
    otree = cKDTree(list(zip(xo, yo)))

    # for all model grid points, get indices of observations within search radius
    inds = otree.query_ball_point(list(zip(xm, ym)), r=sradius)
    
    # initialize array for gridded variable
    grdvar = np.zeros(len(inds))
    
    for gam in gamma:
        for idx, (matches, grd) in enumerate(zip(inds, list(zip(xm, ym)))):

            ocoords = otree.data[matches]

            grdvar[idx] = update_point(grd[1], grd[0], grdvar[idx],
                                       ocoords[:,1], ocoords[:,0],
                                       -InSitu.depth[matches].values,
                                       gam, kappa)
    
    # reshape to 2D array
    grdvar = grdvar.reshape((NEMO.dims['y'], NEMO.dims['x']))
    
    return grdvar    

In [ ]:
# parameters
sradius = 2500   # search radius in meters
gamma = [1., .2] # gamma

grdvar = barnes_gridding(NEMO, InSitu, proj, sradius, gamma)

# create figure        
fig, ax = plt.subplots(1, 1, figsize=(15, 4))
p = ax.imshow(np.ma.masked_values(grdvar, 0.), origin='lower')
plt.colorbar(p)

In [ ]:
# convert lat/lon of NEMO grid and observations to Cartesian coordinates
xm, ym = proj(NEMO.glamt.values.ravel(), NEMO.gphit.values.ravel())
xo, yo = proj(InSitu.lon.values, InSitu.lat.values)

# compute intra-grid distances
z = xm + 1j * ym
Z = np.array([z,] * len(z))

In [ ]:
print(inds)